# Semester Project - Nextbike
## Task 1 - Exploration and Description

In [1]:
from nextbike.preprocessing import Preprocessing 
from nextbike.io import input

import numpy as np

In [2]:
df = input.__read_file()
df.head()

,Unnamed: 0,p_spot,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,b_bike_type,p_name,p_number,p_lng,p_bike
0,0,True,0,2019-01-20 00:00:00,52073,first,7314560,1,51.506613,4,FH-Dortmund Sonnenstraße,7374,7.455587,False
1,1,True,0,2019-01-20 23:59:00,52073,last,7314560,1,51.506613,4,FH-Dortmund Sonnenstraße,7374,7.455587,False
2,2,True,0,2019-01-20 00:00:00,52331,first,113573,2,51.492690,4,Universität/S-Bahnhof,7362,7.417633,False
3,3,True,0,2019-01-20 23:59:00,52331,last,113573,4,51.492690,4,Universität/S-Bahnhof,7362,7.417633,False
4,4,True,0,2019-01-20 00:00:00,31346,first,113543,3,51.523351,4,Brackel Kirche,7337,7.546867,False


### a) The data set shows columns with prefixes p and b. What do you think do they represent? Also try to find good assumptions for the meanings of the columns.

The prefix "p" stands for the <i> positon </i> and the prefix "b" describes the features for the used <i> bike</i> . 

###### Meanings of the columns

| Column      | Description          |
|-------------|----------------------|
|<i> p_spot </i>      |True, if it is an official station                   |
|<i>p_place_type </i>|                      |
|<i>datetime </i></i>    |Datetime of the start or end of a trip |
|<i>b_number </i>    |Bike ID                   |
|<i>trip   </i>      |Values = ["first, last, start, end] <br> defines if a trip starts or ends|
|<i>p_uid </i>       |ID of the bike station / position                      |
|<i>p_bikes </i>     |Number of available bikes at the postion                      |
|<i>p_lat   </i>     |Latitude coordinate of the position                      |
|<i>b_bike_type</i>  |Type of the used bike                      |
|<i>p_name  </i>     |Street or station name of the current position                      |
|<i>p_number  </i>   |ID of the postion / bike station                      |
|<i>p_lng </i>       |Longitude coordinate of the position                      |
|<i>p_bike   </i>    |                      |




### b) The trip column in your data set shows different values. Explain why there are not only two. Are examples with certain values for trip more informative for the analysis of mobility patterns than others?


#### Analyse the trip column

In [3]:
df["trip"].unique()

array(['first', 'last', 'start', 'end'], dtype=object)

There are four different values in the trip column [first, last, start, end]. 
At least two values are required to define whether the dataset belongs to the starting point or the end of the trip. This means that <b> one trip is represented in two successively rows </b> in the dataframe. One of the rows contains the values at the startinging point (i.e. datetime, start position) and the other row contains the values at the ending point of the trip. 

Let's have a deeper look in the dataframe and the trip column.

In [4]:
# there are much more datasets which have the values "start" and "end" in the trip column
df["trip"].value_counts()

start    249536
end      242878
last      88710
first     88528
Name: trip, dtype: int64

In [5]:
df[(df["trip"] == "first") | (df["trip"] =="last")].head(50)

,Unnamed: 0,p_spot,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,b_bike_type,p_name,p_number,p_lng,p_bike
0,0,True,0,2019-01-20 00:00:00,52073,first,7314560,1,51.506613,4,FH-Dortmund Sonnenstraße,7374,7.455587,False
1,1,True,0,2019-01-20 23:59:00,52073,last,7314560,1,51.506613,4,FH-Dortmund Sonnenstraße,7374,7.455587,False
2,2,True,0,2019-01-20 00:00:00,52331,first,113573,2,51.492690,4,Universität/S-Bahnhof,7362,7.417633,False
3,3,True,0,2019-01-20 23:59:00,52331,last,113573,4,51.492690,4,Universität/S-Bahnhof,7362,7.417633,False
4,4,True,0,2019-01-20 00:00:00,31346,first,113543,3,51.523351,4,Brackel Kirche,7337,7.546867,False
5,5,True,0,2019-01-20 23:59:00,31346,last,113543,4,51.523351,4,Brackel Kirche,7337,7.546867,False
6,6,True,0,2019-01-20 00:00:00,50641,first,113561,5,51.506312,4,Hainallee / Südbad,7351,7.470531,False
9,9,True,0,2019-01-20 23:59:00,50641,last,6260019,1,51.493966,4,TU Dortmund Emil-Figge-Straße 50,7367,7.418008,False
10,10,True,0,2019-01-20 00:00:00,53801,first,50383,3,51.513777,14,Westentor,7319,7.455849,False
11,11,True,0,2019-01-20 03:39:00,53801,last,50383,3,51.513777,14,Westentor,7319,7.455849,False


In this filtered dataframe above it gets clear that the examples with the values **first** and **last** in the trip column don't make much sense. Most of the trips in this dataframe have an unlikely long trip duration. The start time of a trip is almost always at 0 AM and the end time of a trip is at 23:59 PM. 
Furthermore the start and the end positions of one trip are the same. 

It could be measurement errors or other data recording errors. <br> 
These datasets can be disregarded for the next steps, because they aren't suitable for further analysis, especially for the preditction of trip durations. 

### c) Based on the given data, create a new DataFrame that stores (at least) the following trip information (“trip format”):
- Bike Number
- Start Time (Either as appropriate data type or as several columns from “Start Month” down to “Start Minute”)
- Weekend (binary)
- Start Position (Either as appropriate data type or as two columns for Longitude and Latitude),
- Duration
- End Time 
- End Position 

In [6]:
df = Preprocessing.get_trip_data()
df.head(5)

,Unnamed: 0_x,datetime_start,b_number,latitude_start,p_name_start,longitude_start,Unnamed: 0_y,datetime_end,latitude_end,p_name_end,longitude_end,trip_duration,coordinates_start,coordinates_end,distance,weekday,weekend,day,month,hour
0,7,2019-01-20 16:22:00,50641,51.506312,Hainallee / Südbad,7.470531,8,2019-01-20 17:00:00,51.493966,TU Dortmund Emil-Figge-Straße 50,7.418008,38,"(51.506311756219, 7.470531463623098)","(51.493965925874996, 7.4180084466934)",3.897290,6,1,20,1,16
1,13,2019-01-20 02:31:00,50425,51.517155,Hauptbahnhof/Bahnhofsvorplatz,7.459931,14,2019-01-20 02:43:00,51.513069,Unionstr.,7.448886,12,"(51.517155427985, 7.459931373596199)","(51.513069322724, 7.448886036872902)",0.891383,6,1,20,1,2
2,19,2019-01-20 11:32:00,53006,51.509557,Ritterhausstr.,7.446949,20,2019-01-20 13:33:00,51.517155,Hauptbahnhof/Bahnhofsvorplatz,7.459931,121,"(51.509557115819995, 7.4469494819641)","(51.517155427985, 7.459931373596199)",1.235649,6,1,20,1,11
3,21,2019-01-20 14:38:00,53006,51.517155,Hauptbahnhof/Bahnhofsvorplatz,7.459931,22,2019-01-20 14:53:00,51.500725,Polizeipräsidium,7.459819,15,"(51.517155427985, 7.459931373596199)","(51.500725323279, 7.4598187208176)",1.827997,6,1,20,1,14
4,23,2019-01-20 17:02:00,53006,51.500725,Polizeipräsidium,7.459819,24,2019-01-20 17:16:00,51.514029,Schwanenwall,7.472570,14,"(51.500725323279, 7.4598187208176)","(51.514028646499995, 7.472570284530001)",1.724677,6,1,20,1,17


#### Adding weather features

The following steps add three weather features to the final trip DataFrame. The ressource for the weather data is "Deutscher Wetterdienst". [Here](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/), you can download the hourly weather data for several cities in Germany. 

The reason why we take the weather data for Waltrop-City is because there is no official weather station directly in Dortmund. There is no data for Dortmund accessable. Waltrop is the closest city to Dortmund, where weather data can be accessed.



In [7]:
df = Preprocessing.get_trip_data(withWeather=True)
df.head(10)

,Unnamed: 0_x,datetime_start,b_number,latitude_start,p_name_start,longitude_start,Unnamed: 0_y,datetime_end,latitude_end,p_name_end,...,coordinates_end,distance,weekday,weekend,day,month,hour,temperature °C,precipitation in mm,precipitation
0,7,2019-01-20 16:22:00,50641,51.506312,Hainallee / Südbad,7.470531,8,2019-01-20 17:00:00,51.493966,TU Dortmund Emil-Figge-Straße 50,...,"(51.493965925874996, 7.4180084466934)",3.897290,6,1,20,1,16,0.5,0.0,0
1,77,2019-01-20 16:42:00,53940,51.507457,Möllerbrücke,7.451364,78,2019-01-20 16:44:00,51.507457,Möllerbrücke,...,"(51.507457007245996, 7.4513643980026)",0.000000,6,1,20,1,16,0.5,0.0,0
2,85,2019-01-20 16:53:00,50061,51.503293,Vinckeplatz,7.455822,86,2019-01-20 17:13:00,51.519332,Cinestar,...,"(51.519331863840996, 7.460124492645298)",1.809251,6,1,20,1,16,0.5,0.0,0
3,91,2019-01-20 16:35:00,51138,51.499039,Steigenberger Hotel / Berswordtstr.,7.451472,92,2019-01-20 16:37:00,51.499039,Steigenberger Hotel / Berswordtstr.,...,"(51.49903890740301, 7.4514716863632)",0.000000,6,1,20,1,16,0.5,0.0,0
4,279,2019-01-20 16:43:00,53120,51.507457,Möllerbrücke,7.451364,280,2019-01-20 17:02:00,51.512836,Am Kaiserbrunnen,...,"(51.512835629153, 7.4822580814362)",2.226714,6,1,20,1,16,0.5,0.0,0
5,363,2019-01-20 16:24:00,53096,51.500675,Kuithanstr.,7.440834,364,2019-01-20 16:40:00,51.510311,Hiltropwall,...,"(51.5103107749, 7.46223704786)",1.832379,6,1,20,1,16,0.5,0.0,0
6,403,2019-01-20 16:05:00,52040,51.502318,Kreuzstraße,7.450029,404,2019-01-20 17:51:00,51.513069,Unionstr.,...,"(51.513069322724, 7.448886036872902)",1.198778,6,1,20,1,16,0.5,0.0,0
7,475,2019-01-20 16:05:00,51426,51.502318,Kreuzstraße,7.450029,476,2019-01-20 17:40:00,51.502318,Kreuzstraße,...,"(51.5023181776, 7.4500286579132)",0.000000,6,1,20,1,16,0.5,0.0,0
8,13,2019-01-20 02:31:00,50425,51.517155,Hauptbahnhof/Bahnhofsvorplatz,7.459931,14,2019-01-20 02:43:00,51.513069,Unionstr.,...,"(51.513069322724, 7.448886036872902)",0.891383,6,1,20,1,2,-5.8,0.0,0
9,319,2019-01-20 02:00:00,53171,51.500675,Kuithanstr.,7.440834,320,2019-01-20 07:15:00,51.500675,Kuithanstr.,...,"(51.500675232617, 7.4408340454102015)",0.000000,6,1,20,1,2,-5.8,0.0,0


In [8]:
Preprocessing.get_write_trip_data(df)

Transformed trip data for Dortmund successfully saved in a csv file!


### d) Calculate the aggregate statistics (i.e., mean and standard deviation) for the trip duration per month, per day of week, and per hour of day. Are there visible differences between weekdays and weekends?

(The differences between weekdays and weekends will be shown in Task 2 by visualizing the data)

#### Calculating aggregate statistic per month, per day of week and per hour of day

##### Statistic per month

In [9]:
# in this array "July" is missing 
month_by_name = np.array(["January", "February", "March", "April", "May", "June", "August", "September", "October", "November", "December"])

# Means per month
df.groupby(['month']).mean()[["trip_duration"]].set_index(keys=month_by_name)

,trip_duration
January,65.135989
February,35.401047
March,41.186327
April,44.510447
May,52.121733
June,42.560769
August,34.758468
September,20.710711
October,18.898973
November,25.229027


In [10]:
# Means per month
# distinguish between weekend and workday
df.groupby(['weekend', 'month']).mean()[["trip_duration"]]

trip_duration
weekend month               
0       1          61.922910
        2          30.756774
        3          35.526872
        4          42.583561
        5          50.622423
        6          41.404373
        8          34.727259
        9          22.114532
        10         20.034604
        11         28.723922
        12         20.083908
1       1          82.441520
        2          79.585526
        3          77.266842
        4          53.505311
        5          58.029577
        6          45.223875
        8          34.858627
        9          16.617877
        10         14.083534
        11         14.396712
        12         13.157236

In [11]:
# Standard deviation per month
df.groupby(['month']).std()[["trip_duration"]].set_index(keys=month_by_name)

,trip_duration
January,131.724449
February,94.228610
March,98.509464
April,99.339453
May,110.840998
June,96.293009
August,79.853703
September,53.338906
October,48.727480
November,66.714969


In [12]:
# Standard deviation per month
# distinguish between weekend and workday
df.groupby(['weekend','month']).std()[["trip_duration"]]

trip_duration
weekend month               
0       1         124.840611
        2          83.261928
        3          86.525411
        4          94.782378
        5         106.765659
        6          90.561902
        8          79.176275
        9          56.939429
        10         50.598669
        11         73.673695
        12         48.771791
1       1         163.037537
        2         158.914616
        3         149.484612
        4         117.911234
        5         125.458601
        6         108.303935
        8          81.999632
        9          40.808317
        10         39.471431
        11         35.610539
        12         43.967446

##### Statistics per day of week

In [13]:
# Means 
weekday_by_name= np.array(["Monday", "Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])
df.groupby(['weekday']).mean()[["trip_duration"]].set_index(weekday_by_name)

,trip_duration
Monday,30.053161
Tuesday,36.014902
Wednesday,33.014809
Thursday,31.970065
Friday,33.902980
Saturday,37.342071
Sunday,32.080302


In [14]:
# Standard deviation 
df[["weekday", "trip_duration"]].groupby("weekday").std().set_index(weekday_by_name)

,trip_duration
Monday,73.128124
Tuesday,86.036889
Wednesday,81.896887
Thursday,80.431986
Friday,80.763024
Saturday,94.889149
Sunday,90.431240


In [15]:
# Means per hour
df.groupby(['hour']).mean()[["trip_duration"]]

,trip_duration
hour,
0,58.854846
1,55.360782
2,33.281008
3,23.742657
4,54.533423
5,65.690177
6,58.079338
7,57.050767
8,45.014704


In [16]:
# Means per hour 
# distinguish between weekend and workday
df.groupby(['weekend','hour']).mean()[["trip_duration"]]

trip_duration
weekend hour               
0       0         54.622517
        1         59.419125
        2         21.605930
        3         20.695751
        4         61.612640
        5         73.246589
        6         56.794803
        7         57.252132
        8         45.370666
        9         39.186665
        10        37.686535
        11        29.819165
        12        32.359621
        13        31.128574
        14        30.482960
        15        27.394074
        16        27.184735
        17        23.230814
        18        24.686730
        19        17.667662
        20        15.388691
        21        10.772230
        22         9.707669
        23         7.260703
1       0         64.934527
        1         46.696288
        2         45.607285
        3         30.374897
        4         27.370274
        5         30.739962
        6         75.176692
        7         55.995294
        8         43.566028
        9         59.547144
        10        55.563372
        11        38.004266
        12        40.752703
        13        40.558159
        14        34.347117
        15        27.789015
        16        32.292039
        17        27.035318
        18        23.971550
        19        20.128543
        20        14.025641
        21        14.486053
        22        10.757914
        23         6.395492

In [17]:
# Standard deviation per hour
df[["hour", "trip_duration"]].groupby("hour").std()

,trip_duration
hour,
0,166.089569
1,145.765061
2,122.470896
3,90.472204
4,148.854879
5,135.961196
6,125.960485
7,118.082078
8,97.729403


In [18]:
# Standard deviation per hour
# distinguish between weekend and workday
df.groupby(['weekend','month']).std()[["trip_duration"]]

trip_duration
weekend month               
0       1         124.840611
        2          83.261928
        3          86.525411
        4          94.782378
        5         106.765659
        6          90.561902
        8          79.176275
        9          56.939429
        10         50.598669
        11         73.673695
        12         48.771791
1       1         163.037537
        2         158.914616
        3         149.484612
        4         117.911234
        5         125.458601
        6         108.303935
        8          81.999632
        9          40.808317
        10         39.471431
        11         35.610539
        12         43.967446